# Building a comprehensive test suite for a simple function (with examples in python)

## Aim: build a comprehensive test suite for a "greatest common divisor" function, `gcd`

> Definition: the **greatest common divisor** (GCD) of two or more integers, which are not all zero, is the largest positive integer that divides each of the integers. 
> For two integers $x$, $y$, the greatest common divisor of $x$ and $y$ is denoted
$\displaystyle \gcd(x, y)$.
>
> For example, $\displaystyle \gcd(8, 12) = 4$.
>
> ... $\displaystyle \gcd(0, 0)$ is commonly defined as $0$ [*and this is our definition today*].
>
> _Source: Wikipedia, https://en.wikipedia.org/wiki/Greatest_common_divisor, retrieved on September 21 2023_

In [55]:
# import our gcd function from our example module
from gcd import gcd as gcd

# Import some other useful utilities
import pytest
import copy

## We want to check for correct functionality with **good data**

### Essential tooling: `assert` lets us throw an error if a check is false

We can use assert statements in python to write tests.

The assert statement in python raises an `Error` if the result of a calculation is "Falsy":

In [56]:
assert True  # no Error

In [57]:
assert False  # raises an Exception

AssertionError: 

In [ ]:
assert 1 == 1  # no Error

In [ ]:
assert 1 > 0  # no Error

In [ ]:
assert 0 > 1  # False, so raises an exception

In [ ]:
assert "a"  # a string is truthy...

In [ ]:
assert ""  # but an empty string is falsy

> Beware: `assert` is meant for debugging, and can be turned off by running `python` with the `-O` flag.
> Use `raise` statements and conditions if your code relies on the check being run.

### Type test

Does it produce sensible results, like the correct datatype?

In [ ]:
assert type(gcd(8, 12)) is int  

... or the correct sign (+ rather than -)?

In [ ]:
assert gcd(8, 12) > 0

### Nominal cases

Check for correct result in "normal", middle-of-the-road cases. 

In [58]:
assert gcd(7, 21) == 7
assert gcd(20, 10) == 10
assert gcd(54, 24) == 6

Ideally, you'd want to test many nominal cases. This could be through calculating them by hand, or constructing examples at random. 

See the section on "property-based testing" for examples of how to do this.

### Boundaries

Check for correctness at the boundaries of the domain, or boundaries within parameters.
Checking the boundary means the value on the boundary, just above, and (if valid) just below.

The `gcd` function operates on integers and has a boundary at zero:

In [59]:
assert gcd(1, 17) == 1  # should be 1
assert gcd(0, 17) == 0  # should be 0

> Python doesn't have a bound on the size of integers, and we'll look at common errors with large values later.

### Compound boundaries

You should test the behavior of your function at places where several variables have boundaries.

In the case of the `gcd`, this is relatively simple:

In [60]:
assert gcd(0, 0) == 0

### Special cases

Check behavior at special values (if any exist):

In [61]:
assert gcd(0, 0) == 0

### Symmetries

We also know that $\gcd(x, y) = \displaystyle \gcd(y, x)$ so we should test those too:

In [62]:
# Nominal
assert gcd(21, 7) == 7
assert gcd(10, 20) == 10
assert gcd(24, 54) == 6

# Boundary
assert gcd(17, 1) == 1  # should be 1
assert gcd(17, 0) == 0  # should be 0

## It is vital to test that our function also throws `Exceptions` correctly for **bad data**



### Uninitialized data

If we pass `None` (where `None` is a disallowed value), it should throw a `TypeError`:

In [63]:
with pytest.raises(TypeError):
    gcd(1, None)

with pytest.raises(TypeError):
    gcd(None, 2)

> Of course, if your function allows `None` as a valid input, it should be included in the **good data** tests. 

### Incorrect type

If we pass in the wrong `type` of data, it should throw a `TypeError`:

In [64]:
with pytest.raises(TypeError):
    gcd(1, 2.4)

with pytest.raises(TypeError):
    gcd(1.2, 2)

with pytest.raises(TypeError):
    gcd(1.2, 2.4)

with pytest.raises(TypeError):
    gcd("one-point-two", 2)

### Too little data

If we pass in too little data it should throw an `Exception`:

In [65]:
with pytest.raises(TypeError):
    gcd()

In [66]:
with pytest.raises(TypeError):
    gcd(0)

### Too much data

If we pass in too little data it should throw an `Exception`:

In [67]:
gcd(1, 2, 3)  # throws a type error

KeyError: 3

In [ ]:
with pytest.raises(TypeError):  # which we can catch like this
    gcd(1, 2, 3)

## **Guess errors** to focus on tests which are disproportionately likely to show problems

Some input values cause more errors than others. 

You might be able to guess which errors will crop up, and test more effectively by finding errors faster.

### Zeros
Zeros often cause problems in numerical functions.

In [ ]:
assert gcd(0, 100) == 0

### Strings: empty, long, unicode

In functions which operate on strings, test the behavior with strings which
- are emtpy,
- are very long compared to the "normal" case in your use case,
- contain unicode characters.

> Strings have a length limit of $(2^{63} - 1)\,\mathrm{B}$ – around $9\,000\,000\,\mathrm{TB}$. 

### Values at the limit of a type's definition may cause issues

The "natural" maximum size of an integer might be $2^{63} - 1$ on a 64-bit system, so we'll check that.

> As of python 3, the only size limit for an integer is the size of memory [[1]](https://docs.python.org/3/library/sys.html#sys.maxsize). 

In [68]:
a = 2**63-1  # prime factors: 7, 73, 127, 337, 92737, 649657, https://www.wikidata.org/wiki/Q10571632
b = 649657 * 7 * 6  # the gcd is 649657 * 7 by construction
assert gcd(a, b) == 649657 * 7

### Mutable datatypes can cause very strange errors

In python, it's easy to introduce a fault which causes function to change its output each time you run it, even with the same inputs – check that a function returns the same output for the same input:

In [69]:
# Example of a function which displays this behavior
def append(value, the_list=[]):
    """
    Appends a value to a list, and if the list isn't given, return the value on a new list
    """
    the_list.append(value)
    return the_list

# Works fine if we give it a list to extend:
append(1, [])  # should return [1]

[1]

In [70]:
assert append(1, []) == [1]

If we don't give it a list to extend, it breaks:

In [71]:
assert append(1) == [1]

In [72]:
assert append(2) == [2]   # should return [2]!!!

AssertionError: 

What's going on? Let's try to debug this function:

In [ ]:
append(2)

The default value of `the_list` is getting extended each time we run the function.

You might think that you can check this by running a test like this:

In [ ]:
assert append(3) == append(3)  # passes unexpectedly!

... but the error is so insidious that this test fails! Both functions are appending to the same list! 
You actually need to store a copy of the value from the first run and compare it later:

In [ ]:
first_result = copy.deepcopy(append(4))
second_result = copy.deepcopy(append(4))

assert first_result == second_result, "%s != %s" % (first_result, second_result)

To fix this, we replace the mutable list in the function with a `None`:

In [ ]:
def append_fixed(value, the_list=None):
    """
    Appends a value to a list, and if the list isn't given, return the value on a new list
    """
    if the_list is None:
        the_list = []
    the_list.append(value)
    return the_list

first_result = copy.deepcopy(append_fixed(4))
second_result = copy.deepcopy(append_fixed(4))

assert first_result == second_result, "%s != %s" % (first_result, second_result)

In the context of our `gcd` function, the test would be:

In [ ]:
first_gcd = copy.deepcopy(gcd(32, 8))
second_gcd = copy.deepcopy(gcd(32, 8))

assert first_gcd == second_gcd, "%s != %s" % (first_gcd, second_gcd)

### Write a **regression test** test for every bug

A rich source of errors is *faults which were already fixed*. If a faults re-emerges, it is called a **regression**.

So, every time you find a bug: 

- Make a test case which fails because of the bug.
- Fix the bug (so the test case passes)
- Leave the test case in your testing library.

## **Property-based testing** helps check more values and locate minimal failing cases

Property-based testing libraries like [`hypothesis`](https://hypothesis.readthedocs.io/): 
- Check that invariant properties of a function are fulfilled for a range of input values.
- "Shrinking" inputs which cause errors systematically to find the "minimal" failing case.

You can convert existing "Example-based" tests into property-based tests.

We start with the basic behavior – the output types and the symmetry between the input values:

In [ ]:
from hypothesis import given, strategies, assume

@given(strategies.integers(), strategies.integers())
def test_gcd_type_symmetry(a, b):
    # Set the boundaries we'll test within. Valid inputs are >= 0
    # Values > 4_000_000 took too long, so limit the upper range.
    assume(0 <= a < 4_000_000 and 0 <= b < 4_000_000)
    
    # Calculate the result
    result = gcd(a, b)

    # Check the type and sign
    assert type(result) is int
    assert result >= 0

    # Check the results are the same when we swap a and b
    result_swapped = gcd(b, a)
    assert result == result_swapped

test_gcd_type_symmetry()

We can also check the degenerate cases.
$\gcd(x, 1) = 1, x \geq 1$:

In [ ]:
@given(strategies.integers())
def test_gcd_type_one(a):
    # Calculate the result
    assume(1 <= a)
    assert gcd(a, 1) == 1

test_gcd_type_one()

$\gcd(x, 0) = 0$:

In [ ]:
@given(strategies.integers())
def test_gcd_type_zero(a):
    # Calculate the result
    assume(0 <= a)
    assert gcd(a, 0) == 0

test_gcd_type_zero()

If you are able to construct cases at random where you know the correct result, then do that too.

We can use the fact that for two numbers the $\gcd$ is the product of the intersection of their 
prime factors (or 1, if they have no matching factors).

We make a way to generate lists of primes (with replacement):

In [ ]:
from functools import reduce
from sympy import primerange

list_of_primes_strategy = strategies.lists(
    strategies.sampled_from(
        list(primerange(0, 30)) 
    ), 
    min_size=0, 
    max_size=30,
    unique=False
)
 
list_of_primes_strategy.example()

We need to find the product of a list of integers:

In [ ]:
def product(x: list[int]):
    if len(x) == 0:
        result = 0
    else:
        result = reduce(lambda x, y: x * y, x, 1)
    return result

# Plausibility checks:
assert product([]) == 0
assert product([1]) == 1
assert product([1, 2]) == 2
assert product([3]) == 3
assert product([3]) == 3
assert product([3, 3, 3]) == 27
assert product([3, 3, 3, 2]) == 54

Here is the product of an example list of primes:

In [ ]:
the_primes = list_of_primes_strategy.example()
print(f"{the_primes=}, {product(the_primes)=}") 

We also need to get the common prime factors from two lists: 

In [ ]:
from collections import Counter

def get_common_elements(ai: list[int], bi: list[int]) -> list[int]:
    # From https://stackoverflow.com/a/37645155, with thanks to "miradulo"
    common_elements = list((Counter(ai) & Counter(bi)).elements())
    return common_elements

# And test it
assert get_common_elements([], []) == []
assert get_common_elements([1], []) == []
assert get_common_elements([], [1]) == []
assert get_common_elements([1], [1]) == [1]
assert get_common_elements([1], [2]) == []
assert get_common_elements([1, 1], [1]) == [1]
assert get_common_elements([1, 1], [1, 1]) == [1, 1]
assert get_common_elements([1, 2], [1, 2]) == [1, 2]
assert get_common_elements([1, 2], [3]) == []
    

Now we can construct as many middle-of-the-road examples as we like:

In [ ]:
@given(list_of_primes_strategy, list_of_primes_strategy)
def test_gcd_constructed_known_cases(a_prime_factors, b_prime_factors):
    # Include 1 in the list of factors to simplify the logic if the list of prime factors is empty
    a_factors = [1] + a_prime_factors
    b_factors = [1] + b_prime_factors
    a, b = product(a_factors), product(b_factors)  
    
    # Skip the testcase if the numbers are too large or negative 
    assume(0 <= a < 4_000_000 and 0 <= b < 4_000_000)
    
    # Get the gcd by construction
    common_factors = get_common_elements(a_factors, b_factors)
    known_gcd = product(common_factors)
    
    # Calculate the result using the function
    calculated_gcd = gcd(a, b) 
    assert calculated_gcd == known_gcd
    
    # Report for debugging purposes
    print(f"{a=}, {b=}, {a_prime_factors=}, {b_prime_factors=}, {common_factors=}, "
          f"{known_gcd=}, {calculated_gcd=}")

test_gcd_constructed_known_cases()

## Organizing tests

### Smoke test

Check for basic plausibility. Does it run without failing?

In [ ]:
gcd(8, 12)  # runs without failing

Does it produce the same result if a and b are swapped? (Only true for commutative operations)

In [ ]:
assert gcd(12, 8)

## The "black-box" approach varies the inputs to a function and checks its outputs

<Image of a black box here>

The first approach we'll use is the "black-box" approach, which treats the function as a black box which we can't see inside. 

We'll look at inputs and check that they produce the correct outputs.